In [1]:
import numpy as np

data = np.load("/home/jkambulo/projects/def-rgmelko/jkambulo/Quantum-Computer-Temperature/data/KZ_Data/12x12/KZ_data_12x12_15_MHz_per_us.npz")
data['rydberg_data'][:,:,-1].shape
# print(data['sweep_rate'])

(12, 12, 1000)

In [3]:
import torch
import numpy as np

def compute_energy(samples: torch.Tensor, logp: torch.Tensor, params: dict, atoms: int = 16):
    """Computes the exact energy based on all the probabilities of each state.

    Args:
        total_configs (torch.Tensor): tensor of all possible states of atoms. Should have shape (2**atoms, atoms)
        logp (torch.Tensor): natural log of the probability of a state, shape (2**atoms,)
        params (dict): Dictionary containing Hamiltonian tuning parameters {'omega', 'delta_per_omega', 'rb_per_a'}
        atoms (int, optional): Number of atoms in grid, assumed to be a perfect square. Defaults to 16.

    Returns:
        float: Calculated energy expectation value
    """
    required_params = {'omega', 'delta_per_omega', 'rb_per_a'}
    assert len(set(params.keys()) ^ required_params) == 0, set(params.keys()) ^ required_params

    C = 862690 * 2 * np.pi
    omega = params['omega']
    rb = (C / omega) ** (1 / 6)
    a = rb / params['rb_per_a']
    delta = params['delta_per_omega'] * omega
    # print(f'c={C} omega={omega} rb={rb} a={a} delta={delta}')

    def compute_interaction(x: torch.Tensor):
        """Computes sum_ij Vij*ni*nj

        Args:
            x (torch.Tensor): tensor of shape (batchsize, natoms)

        Returns:
            torch.tensor: tensor of shape (batchsize,)
        """
        x = x.T
        c = torch.arange(atoms)
        rows = round(np.sqrt(atoms))
        d_inv_6 = torch.triu((1/((c[None]%rows - c[:,None]%rows)**2 +
                                (torch.div(c[None],rows, rounding_mode='floor') - torch.div(c[:,None],rows, rounding_mode='floor'))**2)**3),
                            diagonal=1)
        i, j = torch.triu_indices(atoms, atoms, offset=1)
        filter = (x[i] == 1) & (x[j] == 1)
        return torch.sum(d_inv_6[i[:, None] * filter, j[:, None] * filter], axis=0)

    def compute_rabi(logp):
        L = 2**atoms
        x = torch.arange(L)[:, None]
        i = x ^ (2 ** torch.arange(atoms)[None])  # (2**16, 16)
        return torch.sum(torch.exp(0.5 * (logp[i] + logp[:, None])))

    detuning = -delta * torch.sum(samples, axis=1)
    interaction = C * compute_interaction(samples) / a**6
    rabi_energy = - abs(omega) / 2 * compute_rabi(logp) # - omega to make wavefunction real
    energy = torch.sum(torch.exp(logp) * (detuning + interaction)) + rabi_energy

    return energy.item()

compute_energy(torch.randint(0,2,(16*16,)),np.log(np.ones((16*16,))/256), {'omega':1, 'delta_per_omega':1, 'rb_per_a':1},256)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [1]:
from time import perf_counter
from contextlib import contextmanager


@contextmanager
def catchtime(text) -> float:
    start = perf_counter()
    yield lambda: perf_counter() - start
    print(f'Time: {text} : {perf_counter() - start:.3f} seconds')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchscale.architecture.config import RetNetConfig
from torchscale.architecture.retnet import RetNetDecoder

# vocab size is before embedding. decoder_embed_dim is the number of dimensions in your embedding, ideally less than
# config = RetNetConfig(vocab_size=2, decoder_embed_dim=20, decoder_ffn_embed_dim=1024, decoder_layers=3, decoder_retention_heads=5)
# embedding = nn.Embedding(2, 20)
# linear = nn.Linear(20,2)
# retnet = RetNetDecoder(config, embed_tokens=embedding)
# sum(param.numel() for param in retnet.parameters())
from model_testing.models import RetNet
retnet = RetNet(12,12)
print(sum(p.numel() for p in retnet.parameters()))
retnet


/home/jkambulo/projects/def-rgmelko/jkambulo/py10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


144792


RetNet(
  (retnet): RetNetDecoder(
    (dropout_module): Dropout(p=0.0, inplace=False)
    (embed_tokens): Embedding(2, 12)
    (output_projection): Linear(in_features=12, out_features=2, bias=False)
    (layers): ModuleList(
      (0-4): 5 x DecoderLayer(
        (dropout_module): Dropout(p=0.0, inplace=False)
        (retention): MultiScaleRetention(
          (q_proj): Linear(in_features=12, out_features=12, bias=True)
          (k_proj): Linear(in_features=12, out_features=12, bias=True)
          (v_proj): Linear(in_features=12, out_features=24, bias=True)
          (g_proj): Linear(in_features=12, out_features=24, bias=True)
          (out_proj): Linear(in_features=24, out_features=12, bias=True)
          (group_norm): LayerNorm((8,), eps=1e-05, elementwise_affine=False)
        )
        (retention_layer_norm): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
        (ffn): FeedForwardNetwork(
          (activation_dropout_module): Dropout(p=0.0, inplace=False)
          (d

In [15]:
x = torch.randint(0,2,(30,256))
incremental_state = {}
for i in range(256):
    output, _ = retnet(x[:, i:i+1], incremental_state=incremental_state)
    print(output.shape)
    raise Exception()
    # output = retnet.linear(output)
    # output = retnet.log_softmax(x)
    # print(incremental_state)
print(output.shape)
print(output)
print(torch.argmax(output, dim=-1))
print(output)
print(incremental_state)
with catchtime("parallel"):
    output, _ = retnet.forward(torch.randint(0,2,(200,256)))
# print(output)

torch.Size([30, 1, 2])


Exception: 

In [30]:
import h5py
import numpy as np
# with h5py.File('file.h5', 'w') as f:
#     f['occs'] = np.array([1,2,3,4,5])
    
from contextlib import contextmanager

@contextmanager
def fileloader(filename):
    try:
        if filename.endswith('.jld') or filename.endswith(".h5"):
            file = h5py.File(filename, 'r')
        yield file
    finally:
        file.close()  
    

    
with fileloader("file.h5") as f:
    print(f['occs'])

<HDF5 dataset "occs": shape (5,), type "<i8">


In [3]:
import numpy as np

optimizer = torch.optim.Adam(retnet.parameters(), lr=0.0005, betas=(0.9, 0.98), eps=1e-9)
data = np.load("/home/jkambulo/projects/def-rgmelko/jkambulo/Quantum-Computer-Temperature/data/KZ_Data/12x12/KZ_data_12x12_15_MHz_per_us.npz")['rydberg_data']
data = torch.from_numpy(data[:,:,1].reshape(12*12, -1)).type(torch.int64)

# input = torch.tensor([[0,1,1,0,0,0],
#                       [0,0,0,1,1,0],
#                       [0,0,1,1,0,0],
#                       [0,0,0,0,1,1],
#                       [1,1,0,0,0,0]])
# target = torch.tensor([[0,0,0,1,0,0],
#                        [0,0,0,0,0,1],
#                        [0,0,0,0,1,0],
#                        [0,0,0,0,0,0],
#                        [0,0,1,0,0,0]])
# print(embedding(target).shape)
# loss_fun = torch.nn.NLLLoss()
batchsize = 100
for epoch in range(1,30):
    for i in range(data.shape[-1]//batchsize):
        input = data[:,i*batchsize:(i+1)*batchsize].T
        order = torch.randperm(5)
        optimizer.zero_grad()
        output = retnet(input)
        # print(output)
        # raise Exception('hey')
        # loss = loss_fun(output[order].view(-1, 2), target[order].view(-1))
        loss = - torch.mean(F.one_hot(input, num_classes=2)*output)
        loss.backward()
        optimizer.step()
        print(loss.item())
        del output

0.566020131111145
0.13552944362163544
0.13326728343963623
0.12589557468891144
0.12600383162498474
0.12744538486003876
0.12763060629367828
0.12687455117702484
0.12541240453720093
0.12825848162174225


KeyboardInterrupt: 

In [19]:
retnet(torch.tensor([[0,1,0,0,0,0,0,0,0,0,0]])).exp()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (11x2 and 12x2)

In [6]:
with catchtime('time'):
    retnet.sample(500)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/jkambulo/projects/def-rgmelko/jkambulo/py10/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
  File "/tmp/ipykernel_45752/1939139578.py", line 2, in <module>
  File "/project/6000317/jkambulo/Quantum-Computer-Temperature/code/sweeping-energy/model_testing/models.py", line 85, in sample
  File "/project/6000317/jkambulo/Quantum-Computer-Temperature/code/sweeping-energy/torchscale/torchscale/architecture/retnet.py", line 378, in forward
  File "/home/jkambulo/projects/def-rgmelko/jkambulo/py10/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
  File "/project/6000317/jkambulo/Quantum-Computer-Temperature/code/sweeping-energy/torchscale/torchscale/architecture/retnet.py", line 184, in forward
  File "/home/jkambulo/projects/def-rgmelko/jkambulo/py10/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
  File "/project/6000317/jkambulo/Quantu

In [61]:
# retnet.forward(torch.tensor([1,0,0,0,1,1,0,1,0,1]))
1216/19

64.0